In [1]:
# pip install pytesseract pdf2image opencv-python poppler keras-ocr tensorflow tabula-py

In [2]:
import os
import tempfile
import re

import cv2 as cv2
import pytesseract
import json
import numpy as np
import tabula

from pdf2image import convert_from_path
from PIL import Image

In [3]:
# Initial paths
poppler_path='C://Users//pirun//Downloads//Compressed//poppler-0.68.0_x86//poppler-0.68.0//bin'
pdf_path='C:/Users/pirun/OCR/OCR Task/PDF'
extractedInfo=[]

In [4]:

# convet pdfs and save it in a folder it can handle mul
def ConvertImages(pdf_path,file):
    f=os.path.join(pdf_path,file)
    pages = convert_from_path(f,500,poppler_path=poppler_path)
    
    parent_dir_img= pdf_path.rsplit('/',1)[0]+'/Image/'
    pdf_image_path=parent_dir_img
    if (not os.path.exists(parent_dir_img)):
        os.mkdir(parent_dir_img)
        
    child_dir = pdf_path.rsplit('/',1)[0]+'/Image/'+file.rsplit('.',1)[0]+'/'
    if (not os.path.exists(child_dir)):
        os.mkdir(child_dir)
       
    page_counter =1
    for page in pages:
        filename = "page_"+str(page_counter)+".jpg"
        saveLocation = os.path.join(child_dir,filename)
        page.save(saveLocation,"JPEG")
        
        # parse the doccument
        GetHeaderItems(page_counter , saveLocation)
        
        page_counter=page_counter+1 

In [5]:
pageInfo={}
headerDetails = {}

def GetHeaderItems(page_counter , file_directory):
    pageInfo['Page_Number'] = page_counter
    
    print("Page_Number = " + str(page_counter))
    
    image = Image.open(file_directory)
    print("\n\n")
    RegionTopLeft(image)
    RegionBottomLeftFooter(image)
    RegionTopRight(image)
    RegionFull(image)
    NameRegion(image)
    RegionBody(image)
    addHeaderInformation()
    print("\n\n")
    extractedInfo.append(pageInfo)


def RegionTopLeft(image):
    
    width,height = image.size
    cropString=(0,0,width/2,height/3)
    image = image.crop(cropString)
    text = pytesseract.image_to_string(image)

    patientName = re.search('(Patient)\s*(Name)\s*:\s(.*)', text , re.IGNORECASE)
    if(patientName):
        headerDetails['Patient Name ']=patientName
        print(patientName)

    gstRegNo = re.search(r'GST REG NO\s*:\s*(.*)', text , re.IGNORECASE)
    if(gstRegNo):
        print(gstRegNo)
        
def RegionBottomLeftFooter(image):
    
    width,height = image.size
    cropString=(0,height/10*9,width/3,height)
    image = image.crop(cropString)
    text = pytesseract.image_to_string(image)

    pageOf= re.search('(Page)\s*(.*)', text , re.IGNORECASE)
    if(pageOf):
        print(pageOf) 

    hospitalClinic = re.search(r'(.*)(Hos(.*)|cli(.*))(.*)', text , re.IGNORECASE)
    if(hospitalClinic):
        print(hospitalClinic)
        
def RegionTopRight(image):
    
    width,height = image.size
    cropString=(width/2,0,width,height/3)
    image = image.crop(cropString)
    text = pytesseract.image_to_string(image)

    pageOf= re.search('(Page)\s*(.*)',text , re.IGNORECASE)
    if(pageOf):
        print(pageOf)

    gstRegNo = re.search(r'GST REG NO\s*:\s*(.*)', text , re.IGNORECASE)
    if(gstRegNo):
        print(gstRegNo)

    patientNationalID = re.search(r'\b(.*?NRIC(.*))\b', text , re.IGNORECASE)
    if(patientNationalID):
        print(patientNationalID)

    visitDate = re.search(r'(Visit)\s*(Date)\s*(.*)', text , re.IGNORECASE)
    if(visitDate):
        print(visitDate)

    taxInvoiceDate = re.search(r'(Tax\s*)?(Invoice)\s*(Date)\s*(.*)', text , re.IGNORECASE)
    if(taxInvoiceDate):
        print(taxInvoiceDate)

    BillReceiptDate = re.search(r'((Bill\s*)|(Receipt\s*))(Date)\s*(.*)', text , re.IGNORECASE)
    if(BillReceiptDate):
        print(BillReceiptDate)

    admissionDate = re.search(r'(Admission)\s*(Date)\s*(.*)', text , re.IGNORECASE)
    if(admissionDate):
        print(admissionDate)

    billReferenceNumber = re.search(r'(((Bill\s*)?(Ref|Reference)\s*)|(case\s*))\s*(Number|No)\s*(.*)', text , re.IGNORECASE)
    if(billReferenceNumber):
        print(billReferenceNumber)

    location = re.search(r'(.*)?(Location)\s*(.*)', text , re.IGNORECASE)
    if(location):
        print(location)
        
def RegionFull(image):
    
    text = pytesseract.image_to_string(image)

    billType = re.search('(ORIGINAL|DUPLICATE|INTERIM|Adjusted)', text , re.IGNORECASE)
    if(billType):
        print(billType)
        
    dischargeDate = re.search('([0-3]\s)([0-9]\s)[JFMASOND][AEPUECO][NBRYLGPTVC]\s[2][0-9][0-9][0-9]', text , re.IGNORECASE)
    if(dischargeDate):
        print(dischargeDate)
              
    totalPayable = re.search('(Net|Total)\s(Amount)\s(.*)', text , re.IGNORECASE)
    if(totalPayable):
        print(totalPayable)
    
    doctorName = re.search(r'served by\s(.*)', text)
    if(doctorName):
        print(doctorName)
            
              
def NameRegion(image):
    
    width,height = image.size
    cropString=(width/11,0,width/3.6,height/10*1)
    image = image.crop(cropString)

    text = pytesseract.image_to_string(image)
    hospitalClinic = re.search(r'(.*)(\r?\n)?(.*)(Hos(.*)|cli(.*))', text , re.IGNORECASE)
    if(hospitalClinic):
        print(hospitalClinic)
        
              
              
def RegionBody(image):
    
    width,height = image.size
    cropString=(0,height/4,width,height/4*3)
    image = image.crop(cropString)
    text = pytesseract.image_to_string(image)

    lines = text.split('\n')
    for line in lines:
        match = re.search('(([A-Za-z]|[0-9]|[A-Za-z0-9]|[A-Z0-9]|[a-z0-9]){5,10})\s+(.+)([1-9]{1,3})\s+([0-9]{1,5})(\.|\,)([0-9]{2})',line , re.IGNORECASE)
        
        if(match):
            print(str(match.group()).split(" ",1)[0])
            print(str(match.group()).rsplit(" ",2)[0].split(" ",1)[1])
            print(str(match.group()).rsplit(" ",2)[1])
            print(str(match.group()).rsplit(" ",2)[2])
            print("\n\n")
            
def addHeaderInformation():
    pageInfo["Key_Values"] = headerDetails

In [6]:
def printInfo():
    print(extractedInfo)

In [7]:
for file in os.listdir(pdf_path):
    f=os.path.join(pdf_path,file)
    if os.path.isfile(f):
        ConvertImages(pdf_path,file)
        printInfo()

Page_Number = 1



<re.Match object; span=(45, 68), match='GST REG NO : M90368910N'>
<re.Match object; span=(0, 26), match='Singapore Gencral Hospital'>
<re.Match object; span=(54, 65), match='Page 1 of 2'>
<re.Match object; span=(177, 196), match='Patient NRIC/HRN :S'>
<re.Match object; span=(200, 231), match='Visit Date °22.10.2018 1153 hrs'>
<re.Match object; span=(139, 176), match='Tax Invoice Date :22.10.2018 1546 hrs'>
<re.Match object; span=(102, 138), match='Bill Ref Number -6818337074E-0010-01'>
<re.Match object; span=(233, 275), match='Visit / Bill Location :GCLHAEC / GCLHAEC /'>
<re.Match object; span=(301, 309), match='ORIGINAL'>
<re.Match object; span=(1582, 1594), match='2 8 NOV 2018'>
<re.Match object; span=(1130, 1148), match='NET AMOUNT PAYABLE'>
<re.Match object; span=(1188, 1224), match='served by ZAWIYAH BINTE ABUDULLAH ™*'>
<re.Match object; span=(0, 21), match='_\nfy Ccperal Hospital'>
HLAO3074
HLA DOB TYPING (PCR INT TO HIGH RES)
1
243.93



HF703102
HANDLING FEE

In [8]:
#hold information of all pages
extractedInfo=[]

#hold infomation of a single page
pageInfo={}
pageInfo['Page_Number'] = 1

#hold infomatation of header
headerDetails = {}
headerDetails['BillRefNumber'] = "04D60"
headerDetails['TaxInvoiceDate'] = "7.000.00"

pageInfo["Key_Values"] = headerDetails

extractedInfo.append(pageInfo)


#hold infomation of a single page
pageInfo={}
pageInfo['Page_Number'] = 2

#hold infomatation of header
headerDetails = {}
headerDetails['BillRefNumber'] = "04D6DF0"
headerDetails['TaxInvoiceDate'] = "70.000.00"

pageInfo["Key_Values"] = headerDetails

extractedInfo.append(pageInfo)

print(extractedInfo)

[{'Page_Number': 1, 'Key_Values': {'BillRefNumber': '04D60', 'TaxInvoiceDate': '7.000.00'}}, {'Page_Number': 2, 'Key_Values': {'BillRefNumber': '04D6DF0', 'TaxInvoiceDate': '70.000.00'}}]
